# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [46]:
from xml.etree import ElementTree as ET
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [15]:
document = ET.parse( './data/mondial_database.xml' )

In [16]:
root = document.getroot()

In [17]:
root.tag

'mondial'

### 10 countries with the lowest infant mortality

In [60]:
countries = []
inf_mort_rate = []

In [61]:
for c in root.iter('country'):
    countries.append(c.findtext('name'))
    inf_mort_rate.append(c.findtext('infant_mortality'))

In [67]:
pd_inf_mort_rate = pd.DataFrame({'Infant mortality rate': inf_mort_rate}, index = countries)
pd_inf_mort_rate=pd_inf_mort_rate['Infant mortality rate'].astype('float')
pd_inf_mort_rate = pd_inf_mort_rate.dropna()

In [73]:
pd_inf_mort_rate.order()

Monaco            1.81
Japan             2.13
Bermuda           2.48
Norway            2.48
Singapore         2.53
Sweden            2.60
Czech Republic    2.63
Hong Kong         2.73
Macao             3.13
Iceland           3.15
Italy             3.31
France            3.31
Spain             3.33
Finland           3.36
Anguilla          3.40
...
Equatorial Guinea            71.12
Mozambique                   72.42
Zaire                        73.15
Sierra Leone                 73.29
Nigeria                      74.09
Burkina Faso                 76.80
Angola                       79.99
Niger                        86.27
Chad                         90.30
Guinea-Bissau                90.92
Central African Republic     92.86
Somalia                     100.14
Mali                        104.34
Afghanistan                 117.23
Western Sahara              145.82
Name: Infant mortality rate, Length: 228, dtype: float64